In [6]:
using CollectiveSpins
using PyPlot
using Statistics
using JLD2
using ProgressMeter
using QuantumOptics
using OrdinaryDiffEq

In [7]:
""" Prepare the initial state of the system 
with a phase correlation imposed between the atoms
because the laser arrive on the lattice with an angle θl """
function prepare_phi_IS(θl, λl, theta_array)
    phi_array = zeros(N)
    for i = 1:N
        phi_array[i] = 2pi * sqrt(sum((system.spins[i].position .* [cos(θl), sin(θl), 0]) .^ 2))
    end
    return phi_array
end

""" Prepare the initial vector u0 """
function u0_CFunction(phi_array, theta_array, op_list)
    u0 = ones(ComplexF64, length(op_list))
    for i in 1:length(op_list)
        if length(op_list[i]) == 1
            j = Int(op_list[i][1] % 10^floor(log10(abs(op_list[i][1]))-1)) # Atom nbr
            if parse(Int, string(op_list[i][1])[1:2]) == 22
                u0[i] = cos(theta_array[j]/2)^2
            elseif parse(Int, string(op_list[i][1])[1:2]) == 21
                u0[i] = cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
            else
                println(op_list[i][1])
            end
        end

        if length(op_list[i]) == 2
            for op in op_list[i]
                j = Int(op % 10^floor(log10(abs(op))-1)) # Atom nbr
                if parse(Int, string(op)[1:2]) == 22
                    u0[i] *= cos(theta_array[j]/2)^2
                elseif parse(Int, string(op)[1:2]) == 21
                    u0[i] *= cos(theta_array[j]/2)*exp(1im*phi_array[j])*sin(theta_array[j]/2)
                elseif parse(Int, string(op)[1:2]) == 12
                    u0[i] *= cos(theta_array[j]/2)*exp(-1im*phi_array[j])*sin(theta_array[j]/2)
                else
                    println(op)
                end
            end
        end
    end
    return u0
end

u0_CFunction

### Define the system

In [8]:
# Define geometry of system
Nx,Ny,Nz = [5,5,1]
N = Nx*Ny*Nz
d_xy, d_z = 266., 532. # Optical lattice spacing in nm
e = [0,0,1]   # Quantization axis

# Initial values
theta_init = pi/2
phi_init = 1 # 1 if laser induced correlations between the atoms, 0 else

λ = 1299.
θl_array = range(0, pi/2, 50) # Angle of the laser

a,b,c = [d_xy,d_xy,d_z]/λ
geo = geometry.box(a,b,c;Nx=Nx,Ny=Ny,Nz=Nz)
system = SpinCollection(geo, e, gammas=1.)

Tstep, Tend = 1e-3, 1 # Normalised by Γ0
T = [0:Tstep:Tend;];

### Loop over thetal

In [9]:
@load "op_list.jdl2" op_list
theta_array = ones(N)*theta_init
fsolve(du, u, p, t) = ccall(("diffeqf", "liballfuncs.dll"), Cvoid, (Ptr{ComplexF64}, Ptr{ComplexF64}), du, u)
isdir("Solutions") || mkdir("Solutions")


@showprogress for θl in θl_array
    phi_array = prepare_phi_IS(θl, λ, theta_array)*phi_init;
    u0 = u0_CFunction(phi_array, theta_array, op_list);
    Nvar = length(u0)
    prob = OrdinaryDiffEq.ODEProblem(fsolve, u0, (T[1], T[end]))
    sol = OrdinaryDiffEq.solve(prob, OrdinaryDiffEq.DP5(), saveat=T;
            reltol=1.0e-6,
            abstol=1.0e-8).u;
    @save "Solutions/Sol_QC_Nx_$(Nx)_Ny_$(Ny)_Nz_$(Nz)_theta_$(round(theta_init, digits=1))_phi_$(phi_init)_thetal_$(round(θl, digits=2))" sol
end

Progress: 100%|█████████████████████████████████████████| Time: 0:54:50
